# ATL15 Analysis Demo

In this notebook, we're going to pivot to something more relevant to the IceSAT2 Hackweek. We'll be combining the ATL15 dataset with ERA5.

The goals of the demo are to show participants how to:

1. Use Arraylake access the ATL15 dataset
2. Use Xarray/Zarr/Icechunk to select the region/resolution of interest -- leverging Zarr's hierarchical data structure.
4. Perform a simple statistical analysis, combing ERA5 and ATL15

We'll start by importing the Python libraries we need for the demo:

In [ ]:
# requirements: pip install arraylake rioxarray dask zarr icechunk rich

from arraylake import Client

import zarr
import xarray as xr
from dask.diagnostics import ProgressBar
from rasterio.enums import Resampling
import rioxarray
import matplotlib.pyplot as plt

In [ ]:
client = Client()
client.login()

## ATL15 + ERA5

In this notebook, we'll use part of the ATL15 data product from IceSAT2 to explore changes in ice sheet height. We'll also pull in the ERA5 dataset to add some climatolgical context.

We've already ingested the ATL15 dataset. We'll open it using the Arraylake Client and use Zarr's `tree` api to show us what data is available:

In [ ]:
repo = client.get_repo("ICESAT-2HackWeek/ATL15")
session = repo.readonly_session("main")

root = zarr.open_group(session.store, mode='r')
root.tree()

We'll now open a specific group to perform some basic analysis. 

***Hint: if you are on a small VM, think about using a coarser resolution for this step.***

In [ ]:
# we can open any of these groups using Xarray. 
atl15 = xr.open_zarr(session.store, group="greenland/20km")
atl15

Let's plot one time point from this dataset:

In [ ]:
atl15.isel(time=-20).delta_h.plot(robust=True, figsize=(8,8))

## Exercise 1 -- extract a time series from a single point near the southern tip of Greenland

## Exercise 2 -- Calculate and plot the cumulative change in Ice sheet heigh over the observed record.

## Bonus exercise -- also do this for Antarctica!

(or Greenland if you started with Antarctica)

## Are the changes in Ice sheet height correlated with local climate conditions?

This is a somewhat naive analysis but I hope its useful for showing how we can merge the ERA5 weather reanalysis with the ATL15 dataset.

The steps we'll run through include:

1. Open the ERA5 dataset using the Arraylake Client, Icechunk, and Xarray
2. Sub-select a specific time / region to match ATL15 Greenland
3. Reproject the ERA5 2-meter air temperature dataset onto the ATL15 grid
4. Compare quarterly changes in ice sheet height to temperature

In [ ]:
repo = client.get_repo("earthmover-public/era5-surface-aws")
session = repo.readonly_session("main")
era5 = xr.open_zarr(session.store, group="temporal")
era5

Calculate the Seasonal average 2-meter air temperature for Greenland

Note that we also have some data chores we need to do before we can reproject the data:

- switch the longitude to -180-180
- sort the logitude

In [ ]:
with ProgressBar():
    era5_q = era5[['t2']].sel(time=slice('2019', '2024'), longitude=slice(270, 360), latitude=slice(90, 50)).resample(time='QS-JAN').mean().load()

# fix longitude
era5_q = era5_q.assign_coords(
    longitude=((era5_q.longitude + 180) % 360) - 180
)

# sort so lon increases monotonically
era5_q = era5_q.sortby("longitude")

We can now reproject the data using RasterIO:

In [ ]:
era5_q = era5_q.rio.write_crs("EPSG:4326").rio.set_spatial_dims("longitude", "latitude")
atl15 = atl15.resample(time="QS-JAN").first()
atl15 = atl15.rio.write_crs("EPSG:3413").rio.set_spatial_dims("x", "y")  # or EPSG:3031

era5_q_atl15 = era5_q.rio.reproject_match(atl15, resampling=Resampling.bilinear)
era5_q_atl15.t2.isel(time=0).where(atl15.ice_area.sum(dim='time') > 0).plot()

In [ ]:
# add the reprojected data to the atl15 dataset
atl15['t2'] = era5_q_atl15.t2
atl15

Now that the two datasets are aligned on the same spatio-temporal grid, we look to see if changes in ice sheet thickness are correlated with the seasonal temperature.

**Challenge: can you think of a better analysis? Give it a try 👇**

In [ ]:
atl15.where(atl15.time.dt.season == 'JJA').plot.scatter(y='delta_h', x='t2', hue='delta_h_sigma', alpha=0.7, linewidth=0., vmax=0.5)